In [40]:
#pip install pandas numpy matplotlib tensorflow scikit-learn pandas_datareader

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Carregar os dados
df = pd.read_csv('C:\\Users\\denis\\Estudos\\Estudos_IBM\\Certificado Profissional Engenharia de IA da IBM\\Introdução à aprendizagem profunda e às redes neurais com o Keras\\Dados Históricos FLOKI_USD - LBank.csv')

# Converter a coluna de data para datetime
df['Data'] = pd.to_datetime(df['Data'], format='%d.%m.%Y')

# Substituir vírgulas por pontos e converter as colunas para float
for feature in ['Último', 'Máxima', 'Mínima']:
    df[feature] = df[feature].str.replace(',', '.').astype(float)

# Definir as features e preparar os dados
features = ['Último', 'Máxima', 'Mínima']

# Normalizar dados
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])

# Preparar dados para treinamento
def create_sequences(data, time_step=10):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

time_step = 10  # Número de passos no tempo
X, y = create_sequences(scaled_data, time_step)

# Dividir os dados em treinamento e teste
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Construir o modelo
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, len(features))),
    LSTM(50),
    Dense(len(features))  # Prever todas as features
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1
)

# Avaliar o modelo
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverter a normalização das previsões
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Calcular métricas de erro
mae_train = mean_absolute_error(scaler.inverse_transform(y_train), train_predict)
mse_train = mean_squared_error(scaler.inverse_transform(y_train), train_predict)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(scaler.inverse_transform(y_train), train_predict)

mae_test = mean_absolute_error(scaler.inverse_transform(y_test), test_predict)
mse_test = mean_squared_error(scaler.inverse_transform(y_test), test_predict)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(scaler.inverse_transform(y_test), test_predict)

# Exibir resultados
print(f"MAE (Treinamento): {mae_train}")
print(f"MSE (Treinamento): {mse_train}")
print(f"RMSE (Treinamento): {rmse_train}")
print(f"R² (Treinamento): {r2_train}")

print(f"MAE (Teste): {mae_test}")
print(f"MSE (Teste): {mse_test}")
print(f"RMSE (Teste): {rmse_test}")
print(f"R² (Teste): {r2_test}")

# Prever futuros valores
def predict_future(model, last_data, future_steps, scaler, time_step):
    predictions = []
    data = last_data.copy()
    for _ in range(future_steps):
        pred = model.predict(data[np.newaxis, ...])
        predictions.append(pred[0])
        data = np.vstack([data[1:], pred])
    return np.array(predictions)

# Dados para previsão futura
last_data = scaled_data[-time_step:]
future_steps = 180
future_predictions = predict_future(model, last_data, future_steps, scaler, time_step)

# Inverter a normalização dos dados futuros
future_predictions = scaler.inverse_transform(future_predictions)

# Configurar a exibição de números no formato decimal completo
np.set_printoptions(suppress=True)

# Agora, as previsões futuras serão exibidas sem notação científica
print(f"Previsões futuras para {future_steps} dias:")
print(future_predictions)

Epoch 1/100


C:\Users\denis\anaconda3\envs\IA_env\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - loss: 0.1097 - val_loss: 0.0043
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0164 - val_loss: 0.0030
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0112 - val_loss: 3.0581e-04
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0086 - val_loss: 1.2248e-04
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0068 - val_loss: 2.2435e-04
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0059 - val_loss: 1.7116e-04
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0071 - val_loss: 1.4075e-04
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0063 - val_loss: 1.0384e-04
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0063 - val_loss: 1.1979e-04
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0066 - val_loss: 1.7319e-04
Epoch 11/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0062 - val_loss: 1.0408e-04
Epoch 12/100
12/12 ━━━━━━━━━